In [ ]:
import numpy as np
import random
import nbimporter
from least_squares_M_solver import *
from svd_M_solver import *

In [ ]:
# Predict 2d points by multiplying M and 3d points
def predict_pts(M, actual_pts_3d):
    predicted_pts_2d = np.array([np.dot(M, np.append(pt_3d,1)) for pt_3d in actual_pts_3d])
    # homogeneous 2D point < us, vs, s > to its inhomogeneous version by dividing by s
    predicted_pts_2d = predicted_pts_2d[:,:2] / predicted_pts_2d[:,2:]
    return predicted_pts_2d

# Calculate distance between actual and predicted 2d points
def dist_between(actual_pts_2d, predicted_pts_2d):
    return np.linalg.norm(actual_pts_2d - predicted_pts_2d)

# Calculate M for given number of calibration points for given iterations and select M with minimum residual 
def best_M(pts_2d, pts_3d, num_calibration_pts, num_test_pts, iterations):
    num_pts = pts_2d.shape[0]
    M = np.zeros((3,4), dtype=np.float32)
    residual = 1e9
    for iter in range(iterations):
        idxs = random.sample(range(num_pts), num_calibration_pts)
        M_tmp,_ = least_squares_M_solver(pts_2d[idxs], pts_3d[idxs])
        #  M_tmp = svd_M_solver(pts_2d[idxs], pts_3d[idxs])
        
        test_idxs = set(range(num_pts)) - set(idxs)
        test_idxs = random.sample(test_idxs, num_test_pts)
        
        predicted_pts_2d = predict_pts(M_tmp, pts_3d[test_idxs])
        residual_tmp = dist_between(pts_2d[test_idxs], predicted_pts_2d)
        if residual_tmp < residual:
            residual = residual_tmp
            M = M_tmp
    return M, residual